# Computer Vision Nanodegree

## Project: Image Captioning

---

In this notebook, you will train your CNN-RNN model.  

You are welcome and encouraged to try out many different architectures and hyperparameters when searching for a good model.

This does have the potential to make the project quite messy!  Before submitting your project, make sure that you clean up:
- the code you write in this notebook.  The notebook should describe how to train a single CNN-RNN architecture, corresponding to your final choice of hyperparameters.  You should structure the notebook so that the reviewer can replicate your results by running the code in this notebook.  
- the output of the code cell in **Step 2**.  The output should show the output obtained when training the model from scratch.

This notebook **will be graded**.  

Feel free to use the links below to navigate the notebook:
- [Step 1](#step1): Training Setup
- [Step 2](#step2): Train your Model
- [Step 3](#step3): (Optional) Validate your Model

<a id='step1'></a>
## Step 1: Training Setup

In this step of the notebook, you will customize the training of your CNN-RNN model by specifying hyperparameters and setting other options that are important to the training procedure.  The values you set now will be used when training your model in **Step 2** below.

You should only amend blocks of code that are preceded by a `TODO` statement.  **Any code blocks that are not preceded by a `TODO` statement should not be modified**.

### Task #1

Begin by setting the following variables:
- `batch_size` - the batch size of each training batch.  It is the number of image-caption pairs used to amend the model weights in each training step. 
- `vocab_threshold` - the minimum word count threshold.  Note that a larger threshold will result in a smaller vocabulary, whereas a smaller threshold will include rarer words and result in a larger vocabulary.  
- `vocab_from_file` - a Boolean that decides whether to load the vocabulary from file. 
- `embed_size` - the dimensionality of the image and word embeddings.  
- `hidden_size` - the number of features in the hidden state of the RNN decoder.  
- `num_epochs` - the number of epochs to train the model.  We recommend that you set `num_epochs=3`, but feel free to increase or decrease this number as you wish.  [This paper](https://arxiv.org/pdf/1502.03044.pdf) trained a captioning model on a single state-of-the-art GPU for 3 days, but you'll soon see that you can get reasonable results in a matter of a few hours!  (_But of course, if you want your model to compete with current research, you will have to train for much longer._)
- `save_every` - determines how often to save the model weights.  We recommend that you set `save_every=1`, to save the model weights after each epoch.  This way, after the `i`th epoch, the encoder and decoder weights will be saved in the `models/` folder as `encoder-i.pkl` and `decoder-i.pkl`, respectively.
- `print_every` - determines how often to print the batch loss to the Jupyter notebook while training.  Note that you **will not** observe a monotonic decrease in the loss function while training - this is perfectly fine and completely expected!  You are encouraged to keep this at its default value of `100` to avoid clogging the notebook, but feel free to change it.
- `log_file` - the name of the text file containing - for every step - how the loss and perplexity evolved during training.

If you're not sure where to begin to set some of the values above, you can peruse [this paper](https://arxiv.org/pdf/1502.03044.pdf) and [this paper](https://arxiv.org/pdf/1411.4555.pdf) for useful guidance!  **To avoid spending too long on this notebook**, you are encouraged to consult these suggested research papers to obtain a strong initial guess for which hyperparameters are likely to work best.  Then, train a single model, and proceed to the next notebook (**3_Inference.ipynb**).  If you are unhappy with your performance, you can return to this notebook to tweak the hyperparameters (and/or the architecture in **model.py**) and re-train your model.

### Question 1

**Question:** Describe your CNN-RNN architecture in detail.  With this architecture in mind, how did you select the values of the variables in Task 1?  If you consulted a research paper detailing a successful implementation of an image captioning model, please provide the reference.

**Answer:** CNN-RNN Architecture:

CNN Encoder: The encoder is based on a pre-trained ResNet-18 model. The fully connected (FC) layer of the ResNet model is replaced with a linear embedding layer that reduces the high-dimensional feature vector from the ResNet output to the specified embed_size.
RNN Decoder: The decoder is an LSTM network. It takes the image feature embedding from the CNN as input and generates captions sequentially. The RNN has the following components:
An embedding layer that converts words into vector representations.
An LSTM with a hidden size of hidden_size and a configurable number of layers.
A fully connected (FC) layer that maps the LSTM hidden states to a vocabulary-size output for predicting the next word in the caption.
Selection of Variables:

Batch size: Typically, values like 32 or 64 are used for efficient training on GPUs. I selected a batch size of 64, balancing memory usage and training speed.
Vocab threshold: A threshold of 5 was chosen to ensure that only words appearing frequently in the dataset are included in the vocabulary. This reduces noise from rare words and keeps the vocabulary size manageable.
Vocab from file: False, since I’m creating a new vocabulary based on the dataset.
Embed size: I set embed_size=256, which is a common choice in image captioning models. This size is a trade-off between expressive power and computational efficiency.
Hidden size: hidden_size=512 is a standard choice for the LSTM hidden state size, providing enough capacity for the model to learn meaningful patterns from the captions.
Number of epochs: I selected num_epochs=3 for initial training, which should be sufficient to observe learning behavior before deciding whether to continue for longer.
Save and print frequencies: I set save_every=1 and print_every=100 to ensure regular progress tracking without overwhelming the output.


### (Optional) Task #2

Note that we have provided a recommended image transform `transform_train` for pre-processing the training images, but you are welcome (and encouraged!) to modify it as you wish.  When modifying this transform, keep in mind that:
- the images in the dataset have varying heights and widths, and 
- if using a pre-trained model, you must perform the corresponding appropriate normalization.

### Question 2

**Question:** How did you select the transform in `transform_train`?  If you left the transform at its provided value, why do you think that it is a good choice for your CNN architecture?

**Answer:** I left the transform at its provided value. The default transform_train includes resizing, cropping, and normalization, which are crucial for a CNN. Specifically:

Resizing: Resizing the images to a fixed size ensures consistency in input dimensions for the CNN.
Random cropping: This adds some variability and acts as a form of data augmentation, which helps prevent overfitting.
Normalization: Since a pre-trained ResNet is being used, it's important to normalize the images with the same mean and standard deviation as the original dataset (ImageNet).
These operations make the images compatible with the pre-trained ResNet and provide useful augmentation for training.

### Task #3

Next, you will specify a Python list containing the learnable parameters of the model.  For instance, if you decide to make all weights in the decoder trainable, but only want to train the weights in the embedding layer of the encoder, then you should set `params` to something like:
```
params = list(decoder.parameters()) + list(encoder.embed.parameters()) 
```

### Question 3

**Question:** How did you select the trainable parameters of your architecture?  Why do you think this is a good choice?

**Answer:** I selected the parameters as follows:

`params = list(decoder.parameters()) + list(encoder.embed.parameters())`
This choice was made to:

Freeze the CNN weights: The CNN encoder is based on a pre-trained ResNet model, so I do not update its weights during training. This reduces the computational burden and training time while leveraging the powerful pre-trained features.
Train the embedding and decoder layers: The embedding layer and decoder (LSTM and fully connected layers) are initialized randomly and need to be learned from scratch to map the image features and captions.
This choice ensures that only the newly introduced layers are trained, which is sufficient for the captioning task without requiring the fine-tuning of the entire ResNet model.

### Task #4

Finally, you will select an [optimizer](http://pytorch.org/docs/master/optim.html#torch.optim.Optimizer).

### Question 4

**Question:** How did you select the optimizer used to train your model?

**Answer:** I selected the Adam optimizer; Adam is well-suited for training deep neural networks because it adapts the learning rate for each parameter individually, making it more robust and faster to converge than traditional optimizers like SGD.
Learning rate: A learning rate of 0.001 is a common starting point for Adam. It's high enough to allow quick convergence while avoiding instability during training.

In [1]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/student/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
import torch
import torch.nn as nn
from torchvision import transforms
import sys
sys.path.append('/mnt/disks/legacy-jupytergpu-data/cocoapi/PythonAPI')
from pycocotools.coco import COCO
from data_loader import get_loader
from model import EncoderCNN, DecoderRNN
import math


## TODO #1: Select appropriate values for the Python variables below.
batch_size = 32         # batch size
vocab_threshold = 5      # minimum word count threshold
vocab_from_file = True  # if True, load existing vocab file
embed_size = 256         # dimensionality of image and word embeddings
hidden_size = 512        # number of features in hidden state of the RNN decoder
num_epochs = 3          # number of training epochs
save_every = 1           # determines frequency of saving model weights
print_every = 100       # determines window for printing average loss
log_file = 'training_log.txt'       # name of file with saved training loss and perplexity

# (Optional) TODO #2: Amend the image transform below.
transform_train = transforms.Compose([ 
    transforms.Resize(256),                          # smaller edge of image resized to 256
    transforms.RandomCrop(224),                      # get 224x224 crop from random location
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))])

# Build data loader (TODO: You can change the subset_size, however a 
# very large value would increase training time considerably)
data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         vocab_from_file=vocab_from_file,
                         #subset_size=41000
                        )

# The size of the vocabulary.
vocab_size = len(data_loader.dataset.vocab)

# Initialize the encoder and decoder. 
encoder = EncoderCNN(embed_size)
decoder = DecoderRNN(embed_size, hidden_size, vocab_size, num_layers=1)

# Move models to GPU if CUDA is available. 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder.to(device)
decoder.to(device)

# Define the loss function. 
criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

# TODO #3: Specify the learnable parameters of the model.
params = list(decoder.parameters()) + list(encoder.embed.parameters())

# TODO #4: Define the optimizer.
optimizer = torch.optim.Adam(params, lr=0.001)

# Set the total number of training steps per epoch.
total_step = math.ceil(len(data_loader.dataset.caption_lengths) / data_loader.batch_sampler.batch_size)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.5)


Vocabulary successfully loaded from vocab.pkl file!
loading annotations into memory...
Done (t=1.23s)
creating index...
index created!
Obtaining caption lengths for the subset...


100%|██████████| 414113/414113 [01:07<00:00, 6163.17it/s]
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /home/student/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 242MB/s] 


<a id='step2'></a>
## Step 2: Train your Model

Once you have executed the code cell in **Step 1**, the training procedure below should run without issue.  

It is completely fine to leave the code cell below as-is without modifications to train your model.  However, if you would like to modify the code used to train the model below, you must ensure that your changes are easily parsed by your reviewer.  In other words, make sure to provide appropriate comments to describe how your code works!  

You may find it useful to load saved weights to resume training.  In that case, note the names of the files containing the encoder and decoder weights that you'd like to load (`encoder_file` and `decoder_file`).  Then you can load the weights by using the lines below:

```python
# Load pre-trained weights before resuming training.
encoder.load_state_dict(torch.load(os.path.join('./models', encoder_file)))
decoder.load_state_dict(torch.load(os.path.join('./models', decoder_file)))
```

While trying out parameters, make sure to take extensive notes and record the settings that you used in your various training runs.  In particular, you don't want to encounter a situation where you've trained a model for several hours but can't remember what settings you used :).

### A Note on Tuning Hyperparameters

To figure out how well your model is doing, you can look at how the training loss and perplexity evolve during training - and for the purposes of this project, you are encouraged to amend the hyperparameters based on this information.  

However, this will not tell you if your model is overfitting to the training data, and, unfortunately, overfitting is a problem that is commonly encountered when training image captioning models.  

For this project, you need not worry about overfitting. **This project does not have strict requirements regarding the performance of your model**, and you just need to demonstrate that your model has learned **_something_** when you generate captions on the test data.  For now, we strongly encourage you to train your model for the suggested 3 epochs without worrying about performance; then, you should immediately transition to the next notebook in the sequence (**3_Inference.ipynb**) to see how your model performs on the test data.  If your model needs to be changed, you can come back to this notebook, amend hyperparameters (if necessary), and re-train the model.

That said, if you would like to go above and beyond in this project, you can read about some approaches to minimizing overfitting in section 4.3.1 of [this paper](http://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=7505636).  In the next (optional) step of this notebook, we provide some guidance for assessing the performance on the validation dataset.

In [ ]:
import torch.utils.data as data
import numpy as np
import os
import requests
import time

# Open the training log file.
f = open(log_file, 'w')

accumulation_steps = 4  # Define how many steps to accumulate gradients
optimizer.zero_grad()   # Initialize gradients to zero

for epoch in range(1, num_epochs+1):
    
    for i_step in range(1, total_step+1):
        
        # Randomly sample a caption length, and sample indices with that length.
        indices = data_loader.dataset.get_train_indices()
        # Create and assign a batch sampler to retrieve a batch with the sampled indices.
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler
        
        # Obtain the batch.
        images, captions = next(iter(data_loader))

        # Move batch of images and captions to GPU if CUDA is available.
        images = images.to(device)
        captions = captions.to(device)
        
        # Pass the inputs through the CNN-RNN model.
        features = encoder(images)
        outputs = decoder(features, captions)
        
        # Calculate the batch loss.
        loss = criterion(outputs.view(-1, vocab_size), captions.view(-1))
        loss = loss / accumulation_steps  # Scale the loss by the accumulation steps

        # Backward pass.
        loss.backward()

        if i_step % accumulation_steps == 0 or i_step == total_step:
            # Update the parameters in the optimizer and zero the gradients.
            optimizer.step()
            optimizer.zero_grad()
            
        # Get training statistics.
        stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Perplexity: %5.4f'  % (epoch, num_epochs, i_step, total_step, loss.item() * accumulation_steps, np.exp(loss.item() * accumulation_steps))
        
        # Print training statistics (on same line).
        print('\r' + stats, end="")
        #sys.stdout.flush()
        
        # Print training statistics to file.
        f.write(stats + '\n')
        f.flush()
        
        # Print training statistics (on different line).
        if i_step % print_every == 0:
            print('\r' + stats)
            
    # Save the weights.
    if epoch % save_every == 0:
        torch.save(decoder.state_dict(), os.path.join('./models', 'decoder-%d.pkl' % epoch))
        torch.save(encoder.state_dict(), os.path.join('./models', 'encoder-%d.pkl' % epoch))
    

# Close the training log file.
f.close()

Epoch [1/3], Step [100/12942], Loss: 5.0404, Perplexity: 154.5282
Epoch [1/3], Step [200/12942], Loss: 4.0813, Perplexity: 59.22250
Epoch [1/3], Step [300/12942], Loss: 3.7028, Perplexity: 40.56152
Epoch [1/3], Step [400/12942], Loss: 3.6444, Perplexity: 38.26047
Epoch [1/3], Step [500/12942], Loss: 3.5716, Perplexity: 35.57298
Epoch [1/3], Step [600/12942], Loss: 3.3670, Perplexity: 28.9914
Epoch [1/3], Step [700/12942], Loss: 3.5296, Perplexity: 34.11117
Epoch [1/3], Step [800/12942], Loss: 3.5961, Perplexity: 36.4564
Epoch [1/3], Step [900/12942], Loss: 3.7324, Perplexity: 41.7774
Epoch [1/3], Step [1000/12942], Loss: 3.4522, Perplexity: 31.5710
Epoch [1/3], Step [1100/12942], Loss: 3.2774, Perplexity: 26.50686
Epoch [1/3], Step [1200/12942], Loss: 3.4774, Perplexity: 32.3754
Epoch [1/3], Step [1300/12942], Loss: 3.2581, Perplexity: 25.9990
Epoch [1/3], Step [1400/12942], Loss: 3.1951, Perplexity: 24.4129
Epoch [1/3], Step [1500/12942], Loss: 3.1615, Perplexity: 23.6054
Epoch [1/3],

Epoch [1/3], Step [12500/12942], Loss: 2.1590, Perplexity: 8.66245
Epoch [1/3], Step [12600/12942], Loss: 2.1877, Perplexity: 8.91487
Epoch [1/3], Step [12700/12942], Loss: 2.2129, Perplexity: 9.14254
Epoch [1/3], Step [12800/12942], Loss: 2.2336, Perplexity: 9.33372
Epoch [1/3], Step [12900/12942], Loss: 2.0765, Perplexity: 7.97632
Epoch [2/3], Step [100/12942], Loss: 2.3018, Perplexity: 9.9924415
Epoch [2/3], Step [146/12942], Loss: 2.2596, Perplexity: 9.57888

In [4]:
current_lr = optimizer.param_groups[0]['lr']
print(f"Current learning rate: {current_lr}")

Current learning rate: 0.000125


<a id='step3'></a>
## Step 3: (Optional) Validate your Model

To assess potential overfitting, one approach is to assess performance on a validation set.  If you decide to do this **optional** task, you are required to first complete all of the steps in the next notebook in the sequence (**3_Inference.ipynb**); as part of that notebook, you will write and test code (specifically, the `sample` method in the `DecoderRNN` class) that uses your RNN decoder to generate captions.  That code will prove incredibly useful here. 

If you decide to validate your model, please do not edit the data loader in **data_loader.py**.  Instead, create a new file named **data_loader_val.py** containing the code for obtaining the data loader for the validation data.  You can access:
- the validation images at filepath `'/opt/cocoapi/images/train2014/'`, and
- the validation image caption annotation file at filepath `'/opt/cocoapi/annotations/captions_val2014.json'`.

The suggested approach to validating your model involves creating a json file such as [this one](https://github.com/cocodataset/cocoapi/blob/master/results/captions_val2014_fakecap_results.json) containing your model's predicted captions for the validation images.  Then, you can write your own script or use one that you [find online](https://github.com/tylin/coco-caption) to calculate the BLEU score of your model.  You can read more about the BLEU score, along with other evaluation metrics (such as TEOR and Cider) in section 4.1 of [this paper](https://arxiv.org/pdf/1411.4555.pdf).  For more information about how to use the annotation file, check out the [website](http://cocodataset.org/#download) for the COCO dataset.

In [10]:
import torch
from data_loader_val import get_validation_loader
from model import EncoderCNN, DecoderRNN
from torchvision import transforms
import json

# Set device (GPU/CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def generate_caption(encoder, decoder, image, vocab, max_caption_length=20):
    """
    Generate a caption for the image using the encoder-decoder architecture.
    """
    # Set models to evaluation mode
    encoder.eval()
    decoder.eval()

    # Encode the image
    image_tensor = image.unsqueeze(0).to(device)  # Add batch dimension and move to device
    with torch.no_grad():
        features = encoder(image_tensor)  # Forward pass through the encoder

    # Initialize the caption generation process (start with the start token)
    sampled_ids = [vocab(vocab.start_word)]  # Start token
    inputs = torch.tensor([sampled_ids[-1]]).unsqueeze(0).to(device)  # Add batch dimension

    for _ in range(max_caption_length):
        with torch.no_grad():
            # Forward pass through the decoder
            hiddens, _ = decoder.lstm(inputs, (features, features))  # Assuming LSTM with hidden states
            outputs = decoder.linear(hiddens.squeeze(1))  # Final linear layer
            _, predicted = outputs.max(1)  # Predicted word index

        # Append the predicted word to the list
        sampled_ids.append(predicted.item())

        # If the end token is predicted, stop
        if predicted == vocab(vocab.end_word):
            break

        # Prepare the next input for the decoder (the predicted word)
        inputs = predicted.unsqueeze(0).unsqueeze(0)

    # Convert word indices to words
    predicted_caption = vocab.get_sentence(sampled_ids)  # Ensure vocab has a method to decode word indices
    return predicted_caption

def main():
    # Set up image transformations
    transform = transforms.Compose([ 
        transforms.Resize(256),                    
        transforms.RandomCrop(224),                
        transforms.ToTensor(),                     
        transforms.Normalize((0.485, 0.456, 0.406), 
                             (0.229, 0.224, 0.225))])

    # Load validation data
    val_loader = get_validation_loader(transform=transform, batch_size=1)

    # Load the trained models
    encoder = EncoderCNN()
    decoder = DecoderRNN() 
    encoder = encoder.to(device)
    decoder = decoder.to(device)
    encoder.load_state_dict(torch.load("encoder_3.pkl"))
    decoder.load_state_dict(torch.load("decoder_3.pkl"))

    # Load your vocabulary
    vocab = torch.load('vocab.pkl')  # Ensure you have saved your Vocabulary instance

    # Initialize a dictionary to store the predicted captions
    predictions = []

    # Iterate through the validation dataset
    for i, (image, img_id) in enumerate(val_loader):
        image = image.to(device)
        # Generate predicted caption
        caption = generate_caption(encoder, decoder, image, vocab)
        
        # Save the result in a dictionary
        predictions.append({
            'image_id': img_id.item(),
            'caption': caption
        })

        if (i + 1) % 100 == 0:
            print(f"Processed {i + 1}/{len(val_loader)} images")

    # Save predictions to a JSON file
    output_file = 'predicted_captions.json'
    with open(output_file, 'w') as f:
        json.dump(predictions, f)

    print(f"Saved predictions to {output_file}")

if __name__ == "__main__":
    main()


loading annotations into memory...


FileNotFoundError: [Errno 2] No such file or directory: '/opt/cocoapi/annotations/captions_val2014.json'

In [ ]:
from nltk.translate.bleu_score import sentence_bleu

def calculate_bleu_score(reference_captions, predicted_caption):
    reference_captions = [ref.split() for ref in reference_captions]  # Tokenize references
    predicted_caption = predicted_caption.split()  # Tokenize predicted caption
    
    # Calculate BLEU score
    bleu_score = sentence_bleu(reference_captions, predicted_caption)
    
    return bleu_score
